## Tokenizing Text

LLMs cannot process raw text data directly, since it is categorical and so isn't compatible with the mathematical operations used to train neural networks. Therefore, we need a way to represent words as continuous-valued vectors. This is known as <i>embedding</i>. 

The steps involved in embedding include splitting a text into words, converting those words into tokens, and then turning those tokens into embedding vectors.

The goal here will be to tokenize the 20,479-character short story into individual words and special characters, that can then be turned into embeddings for LLM training.

In [3]:
# Get short story to work with
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7f9328461a30>)

In [5]:
# First 100 characters
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters: ", '{:,}'.format(len(raw_text)))
print(raw_text[:99])

Total number of characters:  20,479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


As a first pass, we can use the re.split command to split on whitespace characters '(\s)'. To also split on commas and full stops, we add '([,.])'.

In [7]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [8]:
# To also remove the redunant whitespace characters
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


We can now apply this to the short story, modifying it to work with other forms of punctuation.

In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30]) # First 30 characters

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### Converting tokens into token IDs

We convert the tokens from a Python string into an integer representation to produce the token IDs. This is an intermediate step before converting the token IDs into embedding vectors.

In [10]:
# Create a list of all unique tokens and sort them alphabetically
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [12]:
# Print first 21 entries
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


We can build a tokenizer class with an <i>encode</i> method that splits text into tokens and carries out the string-to-integer mapping to produce token IDs via the vocabulary. We will also implement a <i>decode</i> method that carries out the reverse integer-to-string mapping to convert the token IDs back into text. 

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab # vocab stored as class attribute for access in encode & decode methods
        self.int_to_str = {i:s for s,i in vocab.items()} # mapping for token IDs -> original text tokens

    def encode(self, text): # input text -> token IDs
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids): # convert token IDs -> text
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) # remove spaces before the specified punctuation
        return text